In [30]:
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

quandl.ApiConfig.api_key = '3qNsf3VUNsvytS7sS_7W'
selected = ['TSLA', 'FB', 'MSFT', 'GE', 'AAPL']
data = quandl.get_table('WIKI/PRICES', ticker = selected, qopts = { 'columns': ['date', 'ticker', 'adj_close'] }, date = { 'gte': '2018-1-1', 'lte': '2018-12-31' }, paginate=True)

clean = data.set_index('date')
table = clean.pivot(columns='ticker')

# Calculate daily & annual returns of the stocks
returns_daily = table.pct_change()
returns_annual = returns_daily.mean() * 253

print(returns_annual.head())

# Lists to store portfolio attributes
port_returns = []
port_volatility = []
stock_weight = []

# set the number of combinations for imaginary portfolios
num_assets = len(selected)
num_portfolio = 50000

# Populate empty lists with each port's attributes
for port in range(num_portfolio):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    returns = 0


           ticker
adj_close  AAPL     -0.065209
           FB       -0.707622
           GE       -1.204489
           MSFT      0.224700
           TSLA     -0.515517
dtype: float64
